# Data cleaning 

In [91]:
import pandas as pd

In [92]:
from use_cases.emotions import process_emotions
from use_cases.contributions import process_contributions

from use_cases.personal_needs import process_needs

from use_cases.dialogues import create_table_dialogues
from use_cases.country_needs import create_table_country_needs
from use_cases.persons import create_table
from use_cases.individuals import create_table_individuals


pd.options.mode.chained_assignment = None 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data Surveys

In [274]:
chunksize   = 100 # Just for debugging
survey_path = './data/BBDD_Dialogos_2021_01_27.csv' 
survey = pd.read_csv(survey_path, chunksize=chunksize, low_memory=False)
for survey in survey: break
    
ind_online_survey_path = './data/Consulta_Individual_online_v2.xlsx'
ind_online_survey = pd.read_excel(ind_online_survey_path, 'Sheet1')
    
ind_survey_path = './data/Base_final_digitación_Consultas.xlsx'
ind_survey = pd.read_excel(ind_survey_path, 'CONSULTAS')

In [94]:
survey.sample(2)
ind_online_survey.sample(2)

,Submission Date,RUN,Edad,Comuna,1. ¿Cuál es el nivel de educación alcanzado por Usted?,2. ¿Cuál es el nivel de educación que alcanzó el jefe de su hogar? (sólo si se trata de alguien distinto a Usted),1 >> Emociones / Sentimientos / Sensaciones,1 >> Explique lo mencionado,2 >> Emociones / Sentimientos / Sensaciones,2 >> Explique lo mencionado,...,2 >> Rol del actor social (Describa),3 >> Necesidades del país identificadas,3 >> Rol del Estado (Describa),"3 >> Actor social (empresa, organizaciones sociales, medios de comunicación, comunidad, etc)",3 >> Rol del actor social (Describa),5. Pregunta: ¿Cuál es mi contribución personal para construir el Chile que queremos?,Comuna.1,"Comentarios finales: Si tiene alguna observación o sugerencia, en este espacio puede indicarla.",Región,Región.1
204,2020/01/24 08:46:14,bfadfcf50714ca24c5ca30ec55385148,34,NaN,Universitaria completa.,Yo soy el jefe de hogar.,Preocupado,Me preocupa el uso político de las manifestaci...,Avergonzado,Da vergüenza ver cómo se muestra nuestro país ...,...,El principal responsable de la educación de un...,Ciudades más equitativas,Distribuir mejor recursos y generar más recursos,Comunidad,Terminar con los estigmas de ciertos barrios y...,Aportar con información al debate de cómo cons...,Las Condes,El principal problema de hoy pasa por la educa...,Región Metropolitana de Santiago,NaN
1221,2019/12/19 00:47:44,7183ff22a93fb6f17855786119363bc8,18,Los Andes,Universitaria incompleta. Técnica completa.,Universitaria incompleta. Técnica completa.,Pena,Saqueos,Preocupación,Saqueos a pymes,...,Libre empresa,NaN,NaN,NaN,NaN,Práctica laboral gratuita a pyme con problemas...,NaN,"Liberalizar mercado, respetar derechos de prop...",NaN,V Región de Valparaíso


## Processing Uses Cases
1. Emotions 
2. Contributions
3. Country needs 
4. Personal/familiar needs

### Persons Table (Dialogue)

In [22]:
persons_table = create_table(survey)

In [24]:
persons_table.sample(3)

,file_id,age,sex,level,comuna,age_range
39,enc_u_4598379130154541284,39,h,basica_completa,talca,30-45
0,enc_u_4602780640112847056,54,f,basica_incompleta,,45-60
59,enc_u_4597643380118026494,47,f,basica_completa,temuco,45-60


### Dialogues

In [176]:
dialogues_table = create_table_dialogues(survey)

In [28]:
dialogues_table.sample(3)

,file_id,date,init_time,end_time,location,address,comuna,n_members,group_name
37,enc_u_4598383170151924860,11-02-2020,14:50:00,16:25:00,casa,villa magisteco,talca,5,usuarios de oficina movil de bienes nacionales
83,enc_u_4596763700997702847,09-03-2020,18:30:00,20:30:00,barros luco y vecinos,barros luco #2247,arica,5,usuarios de oficina movil de bienes nacionales
62,enc_u_4597641730116574573,05-03-2020,10:00:00,12:00:00,centro cultural de freire,pedro canales numero 137,freire,5,usuarios de oficina movil de bienes nacionales


### Emotions

<img src="diagrams/q1.png" width=730 height=520 />

In [29]:
%%time
emotions_table = process_emotions(survey)

CPU times: user 994 ms, sys: 119 ms, total: 1.11 s
Wall time: 1.12 s


In [31]:
emotions_table.sample(2)

,file_ids,name,name_token,macro,text,text_tokens
71,ENC_U_4597419179157584588,impotencia,[impotencia],impotencia,de que la justicia no haga lo suyo.,"[justicia, haga]"
9,ENC_U_4602015808803327609,dolor,[dolor],dolor,por sentirse abandonado,"[sentirse, abandonado]"


### Individuals

In [275]:
individuals_table = create_table_individuals(ind_online_survey, ind_survey)

In [287]:
individuals_table.sample(2)

,id,date,age,comuna,level,online
784,2588,17-01-2020,37,10104,Media completa. Técnica incompleta,False
4345,6149,27-01-2020,65,5703,Media incompleta (incluyendo Media Técnica),False


### Country Needs

In [41]:
n = create_table_country_needs(survey)

0                  salud
1                  salud
2     mejores pensiones.
3       pensiones dignas
4                  salud
             ...        
95                   afp
96      mejorar la salud
97        medio ambiente
98                 salud
99            educacion 
Name: P2_1_A, Length: 100, dtype: object


### Family Needs

### Contributions